In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [2]:
comp = 'paddy-disease-classification'
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')
from fastai.vision.all import *
set_seed(42)

tst_files = get_image_files(path/'test_images').sorted()

In [3]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: label, dtype: int64

In [5]:
n=1000
d = df.sample(n=n, ignore_index=True)
d.label.value_counts()

normal                      167
blast                       160
hispa                       139
dead_heart                  136
tungro                      103
brown_spot                   93
downy_mildew                 66
bacterial_leaf_blight        54
bacterial_leaf_streak        44
bacterial_panicle_blight     38
Name: label, dtype: int64

In [4]:
trn_path = path/'train_images'

In [7]:
def get_images(trn_path):
    return [Path(trn_path/d.label[i]/d.image_id[i]) for i in range(n)]

In [8]:
def create_model(arch, size=128, item=Resize((192,144)), accum=1):
    dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y = parent_label,
    splitter=RandomSplitter(0.2, seed=42),
    item_tfms=item,
    batch_tfms=aug_transforms(size=size, min_scale=0.75)
).dataloaders(trn_path, bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    return learn

def train_model(model, epochs=12, lr=0.01, test=True):
    model.fine_tune(epochs, lr)
    if test:
        return model.tta(dl=model.dls.test_dl(tst_files))
    else:
        return model.tta(dl=model.dls.valid)

In [9]:
res = 640, 480
models = {
    'convnext_large_in22k': {
        (Resize(res), 224),
        (Resize(res), (320,224)),
    },
    'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    },
    'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 
    'swin_large_patch4_window7_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }
} 

In [10]:
import gc

In [10]:
tta_res = pd.read_pickle('tta_res1.pkl')
i = 1

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        learn = create_model(arch, size, item=item, accum=2)
        tta_res.append(train_model(learn, epochs=12, lr=0.01, test=True)) #, epochs=1))
        if i==0:
            learn.save(f'test_{arch}1')
            i = 1
        else:
            learn.save(f'test_{arch}2')
            i = 0
        save_pickle('tta_res1.pkl', tta_res)
        gc.collect()
        torch.cuda.empty_cache()

--- swinv2_large_window12_192_22k
192
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


/root/mambaforge/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v2.0.0/swinv2_large_patch4_window12_192_22k.pth" to /root/.cache/torch/hub/checkpoints/swinv2_large_patch4_window12_192_22k.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.090609,0.622559,0.203748,06:17


epoch,train_loss,valid_loss,error_rate,time
0,0.500437,0.251413,0.086977,08:07
1,0.369861,0.240946,0.075925,08:07
2,0.372103,0.347689,0.106679,08:07
3,0.296939,0.236055,0.065834,08:09
4,0.266835,0.162316,0.049495,08:10
5,0.225459,0.144079,0.043729,08:07
6,0.123975,0.118931,0.033157,08:08
7,0.085734,0.089564,0.024988,08:08
8,0.072284,0.085797,0.025469,08:08
9,0.043618,0.080183,0.020183,08:07


--- swin_large_patch4_window7_224
224
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


...

epoch,train_loss,valid_loss,error_rate,time
0,1.110954,0.594423,0.184527,06:34


epoch,train_loss,valid_loss,error_rate,time
0,0.518444,0.259050,0.080730,08:38
1,0.432557,0.227170,0.072561,08:38
2,0.349372,0.243839,0.074003,08:37
3,0.294939,0.206267,0.067756,08:38
4,0.248286,0.157880,0.050457,08:38
5,0.219289,0.160073,0.044690,08:38
6,0.127147,0.137574,0.037001,08:38
7,0.112419,0.105112,0.028832,08:38
8,0.069517,0.087789,0.021144,08:38
9,0.056175,0.081955,0.022585,08:38


--- swin_large_patch4_window7_224
224
Resize -- {'size': (480, 480), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,1.128288,0.757440,0.222489,06:35


epoch,train_loss,valid_loss,error_rate,time
0,0.539374,0.308706,0.100913,08:35
1,0.409919,0.259296,0.086497,08:35
2,0.362493,0.324110,0.098991,08:35
3,0.303189,0.179730,0.057184,08:35
4,0.245772,0.191493,0.060067,08:35
5,0.181229,0.144669,0.038924,08:35
6,0.110310,0.104744,0.030274,08:35
7,0.113122,0.094284,0.027871,08:35
8,0.084020,0.094949,0.027871,08:35
9,0.052531,0.080129,0.023066,08:35


In [10]:
tta_res = pd.read_pickle('tta_res1.pkl')
tta_prs = first(zip(*tta_res))

In [24]:
tta_prs += tta_prs[2:4]

In [25]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

torch.Size([3469, 10])

In [10]:
tta_valid = []#pd.read_pickle('tta_valid1.pkl')
i = 1

for arch,details in models.items():
    for item,size in details:
        dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=20, item_tfms=item, batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//2)
        learn = vision_learner(dls, arch, metrics=error_rate, cbs=GradientAccumulation(64)).to_fp16()
        learn.load(f'test_{arch}{i}')
        if i==1: 
            i = 2
        else:
            i = 1 
        preds, targ = learn.tta()
        tta_valid.append(preds)
        save_pickle('tta_valid3.pkl', tta_valid)

/root/mambaforge/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [39]:
targ = []

for s in [42, 4, 30, 20]:
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=s, item_tfms=[Resize((640,480))], 
                                       batch_tfms=aug_transforms(size=480, min_scale=0.75), bs=32)
    learn = vision_learner(dls, 'resnet18', metrics=error_rate, cbs=GradientAccumulation(64)).to_fp16()
    preds, t = learn.tta()
    targ.append(t)

In [41]:
res = []
tta_valid = pd.read_pickle('tta_valid.pkl')
res.append(torch.stack([error_rate(tta_valid[i], targ[0]) for i in range(8)]))

for n in range(1,4):
    tta_valid = pd.read_pickle(f'tta_valid{n}.pkl')
    res.append(torch.stack([error_rate(tta_valid[i], targ[n]) for i in range(8)]))
    
# seeds: 42, 4, 30, 20

In [49]:
idx = [r.argsort() for r in res]

In [59]:
sort = [r/r.max() for r in res]
sort = np.array([s.tolist() for s in sort])

In [68]:
ssum = sort.sum(0)
ssum/ssum.max(), ssum.argsort()

(array([0.83421587, 0.92386785, 1.        , 0.91921271, 0.94096857,
        0.91089069, 0.90503019, 0.97476481]),
 array([0, 6, 5, 3, 1, 4, 7, 2]))

In [106]:
tta_valids = [pd.read_pickle(f'tta_valid{n}.pkl') for n in range(1,5)]

def calc_error(tta, targ):
    for n in range(4):
        print(torch.stack([error_rate(tta[n][i], targ[n]) for i in range(8)]))
        
calc_error(tta_valids, targ)

TensorBase([0.0168, 0.0221, 0.0221, 0.0202, 0.0163, 0.0226, 0.0187, 0.0197])
TensorBase([0.0058, 0.0115, 0.0120, 0.0096, 0.0111, 0.0096, 0.0101, 0.0096])
TensorBase([0.0067, 0.0043, 0.0053, 0.0067, 0.0058, 0.0038, 0.0058, 0.0072])
TensorBase([0.0082, 0.0077, 0.0086, 0.0067, 0.0091, 0.0091, 0.0077, 0.0082])


In [96]:
tta_res = pd.read_pickle('tta_res1.pkl')
tta_prs = first(zip(*tta_res))
#tta_prs += (tta_prs[0], tta_prs[0])
print(len(tta_prs))
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

8


torch.Size([3469, 10])

In [90]:
tta_res = pd.read_pickle('tta_res1.pkl')
tta_prs = first(zip(*tta_res))
avg_pr = tta_prs[0]

In [97]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)

In [98]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'big model ensemble', comp)

100%|██████████| 70.4k/70.4k [00:00<00:00, 160kB/s]
